## VAISHNAVI JAMDADE(TM39453)
### HOMEWORK 4

### Importing required libraries:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import idx2numpy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

### Converting the file into numpy array of dimensions (60000,28,28)

In [2]:
def convertfunc(file):
    arr = idx2numpy.convert_from_file(file)
    return arr

In [3]:
X_train=convertfunc('train-images.idx3-ubyte')
y_train=convertfunc('train-labels.idx1-ubyte')
X_test=convertfunc('t10k-images.idx3-ubyte')
y_test=convertfunc('t10k-labels.idx1-ubyte')

In [4]:
X_train.shape

(60000, 28, 28)

In [5]:
X_test.shape, y_train.shape

((10000, 28, 28), (60000,))

### Reshaping the dataset :

In [6]:
def reshapefunc(array,rows,columns):
    array=array.reshape(rows,columns)
    return array

In [7]:
X_train=reshapefunc(X_train,60000,784)
X_test=reshapefunc(X_test,10000,784)
#y_train=reshapefunc(y_train,1,60000)
#y_test=reshapefunc(y_test,1,10000)

In [8]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

((60000, 784), (10000, 784), (60000,), (10000,))

### Splitting Training data into Training and Validation dataset

* For Question 3, Here we have split the original X_train(training dataset) into X_trainval and X_testval where we will train our model with 4 configurations on our X_trainval data, and then evaluate it on our Validation dataset that is, X_testval

In [9]:
X_trainval,X_testval,y_trainval,y_testval=train_test_split(X_train,y_train,test_size=0.3, random_state=42)

In [10]:
X_trainval.shape, X_testval.shape, y_trainval.shape,y_testval.shape

((42000, 784), (18000, 784), (42000,), (18000,))

### Using One hot label encoding for output labels:

In [11]:
y_train.shape

(60000,)

In [12]:
y_train = pd.get_dummies(y_train)
y_train=np.array(y_train)

In [13]:
y_train.shape

(60000, 10)

### Using one hot label encoding for validation output labels:

In [14]:
y_trainval = pd.get_dummies(y_trainval)
y_trainval=np.array(y_trainval)

In [15]:
y_trainval.shape

(42000, 10)

In [16]:
y_train.shape

(60000, 10)

### Initializing weights and bias:

In [17]:
def initialize_weights(inputs, hNeurons, oNeurons):

    weights1=np.random.randn(inputs,hNeurons)*np.sqrt(1./inputs)    #784 rows 5 columns
    bias1=np.zeros((1, hNeurons))*np.sqrt(1./inputs)
    #hence x.w = 1,5 dimensions + bias [1,5]
    weights2=np.random.randn(hNeurons,oNeurons)*np.sqrt(1./hNeurons)
    #hence x.w= 5,10 dimensions + bias [1,10]
    bias2= np.zeros((1, oNeurons))*np.sqrt(1./hNeurons)

    #ouptut vecotr is going to be of dimensions [1,10] then softmax
    #W1.shape , b1.shape, W2.shape, b2.shape
    
    return weights1, bias1, weights2, bias2


### Sigmoid Activation Function:

In [18]:
def sigmoid(z):
    act = 1. / (1. + np.exp(-z))
    return act

### Softmax Activation function :

In [31]:
#Normalising the values:

def softmax(z):
    softact = np.exp(z)/np.sum(np.exp(z), axis=1,keepdims=True)                                       
    return softact


### Feed Forward Function :

In [20]:
def feedforward(X_t,W1,b1,W2,b2):
    z1= np.dot(X_t,W1)+b1
    h1=sigmoid(z1)
    
    z2=np.dot(h1,W2)+b2
    output=softmax(z2)
    return z1, h1, z2, output
        

### Error Function :

In [21]:
def error(ypred,y_t):     
    n_samples = y_t.shape[0]
    L = (ypred-y_t)/n_samples
    return L    

### Cross Entropy Loss Function:

In [22]:
def crossentropy(ypred,y_t):
    L_sum = np.sum(np.multiply(y_t, np.log(ypred)))
    num_samples = y_t.shape[0]
    L = -(1./num_samples) * L_sum

    return L

### Function to calculate Sigmoid Derivative:

In [23]:
def sigmoid_derivative(h1):
    deriv=h1*(1-h1)
    return deriv

### Backpropagation Function to compute Gradient Information:

In [24]:
def backpropagation(h1,W2,ypred,y_t,X_t):
    
    #Calculate error
    L=error(ypred,y_t)
    
    # z1, h1, z2, output
    # z1, and z2 => dot products
    # h1 and output => activations
    change_output=L
    #dL/dW2
    delta_W2= np.dot(h1.T,change_output)
    
    #dL/db2
    delta_b2= np.sum(change_output,axis=0,keepdims=True)
    
    #dL/dh1
    delta_h1=np.dot(change_output,W2.T)

    change_h1= delta_h1*sigmoid_derivative(h1)
    
    #dL/dW1
    delta_W1= np.dot(X_t.T, change_h1)
    
    #dL/db1
    delta_b1= np.sum(change_h1,axis=0,keepdims=True)
    
    return delta_W2, delta_b2, delta_W1, delta_b1
    

### Function to update weights:

In [25]:
def update_weights(alpha,W1,b1,W2,b2,delta_W2,delta_b2,delta_W1,delta_b1):
    W2= W2- alpha*(delta_W2)
    b2= b2 - alpha*(delta_b2)
    W1= W1- alpha*(delta_W1)
    b1=b1- alpha*(delta_b1)
    
    return W2,b2,W1,b1

### Training Function:

In [26]:
def training(X_t,y_t,neurons,lr,iterations):
    hNeurons=neurons                              #hidden neuron
    inputs=X_t.shape[1]                   #number of inputs
    oNeurons=y_t.shape[1]
    alpha= lr                       #learning rate
    epochs=iterations
    
    #retriving the actual labels for y_train:
    labely_train=np.argmax(y_t,axis=1)
    
    #initialize weights
    W1, b1, W2, b2= initialize_weights(inputs,hNeurons,oNeurons)
    
    for i in range(epochs):
        
        #feed forward
        dotprod1, h1, dotprod2, output=feedforward(X_t,W1,b1,W2,b2)

        #back propogate
        delta_W2, delta_b2, delta_W1, delta_b1= backpropagation(h1,W2,output,y_t,X_t)
        
        #update weights using gradient descent
        W2,b2,W1,b1=update_weights(alpha,W1,b1,W2,b2,delta_W2,delta_b2,delta_W1,delta_b1)

        #again feed forward using updated weights
        dotprod1, h1, dotprod2, output=feedforward(X_t,W1,b1,W2,b2)

        #Calculating Total Loss
        loss=crossentropy(output,y_t)                 # put in training func later
        print("Epoch {}: training loss = {}".format(i + 1,loss))
        

        #Retrieving the corresponding class labels:
        labelpred=np.argmax(output,axis=1)
        
        #Training Accuracy:
        accuracy = round((accuracy_score(labely_train, labelpred)*100),2)
        print("Training Accuracy after Epoch {} : {}%".format(i+1, accuracy))
        
    return W2,b2,W1,b1
        


### Testing Function:

In [27]:
def testing(W2,b2,W1,b1,X_t,y_t):
    
    dotprod1, h1, dotprod2, output=feedforward(X_t,W1,b1,W2,b2)   
    labely_test=y_t
    labelpred=np.argmax(output,axis=1)
    #accuracytest = round((accuracy_score(labely_test, labelpred)*100),2)        
    #print("Accuracy on Testing data : {}%".format(accuracytest))
    return labely_test,labelpred
        

### Function to measure accuracy:

In [28]:
def accuracy(labely_test, labelpred):
    accuracytest = round((accuracy_score(labely_test, labelpred)*100),2)        
    #print("Accuracy : {}%".format(accuracytest))
    return accuracytest

### Configuration 1 :
* Learning rate = 0.01
* Number of neurons = 128
* Iteratations =100

In [29]:
W2,b2,W1,b1=training(X_trainval,y_trainval,128,0.01,100)

Epoch 1: training loss = 2.4517242899731873
Training Accuracy after Epoch 1 : 11.18%
Epoch 2: training loss = 2.4135126899032304
Training Accuracy after Epoch 2 : 11.95%
Epoch 3: training loss = 2.3778329151156643
Training Accuracy after Epoch 3 : 12.77%
Epoch 4: training loss = 2.343561890589584
Training Accuracy after Epoch 4 : 13.92%
Epoch 5: training loss = 2.3114409968589125
Training Accuracy after Epoch 5 : 15.1%
Epoch 6: training loss = 2.281062432146635
Training Accuracy after Epoch 6 : 16.31%
Epoch 7: training loss = 2.2518189143622425
Training Accuracy after Epoch 7 : 17.86%
Epoch 8: training loss = 2.2241898628295833
Training Accuracy after Epoch 8 : 19.37%
Epoch 9: training loss = 2.197807278531044
Training Accuracy after Epoch 9 : 20.96%
Epoch 10: training loss = 2.1722232291726744
Training Accuracy after Epoch 10 : 22.64%
Epoch 11: training loss = 2.147833475981197
Training Accuracy after Epoch 11 : 24.42%
Epoch 12: training loss = 2.1244679846210976
Training Accuracy aft

Epoch 96: training loss = 1.2132471369402602
Training Accuracy after Epoch 96 : 75.6%
Epoch 97: training loss = 1.2079375693989145
Training Accuracy after Epoch 97 : 75.75%
Epoch 98: training loss = 1.2026723427797898
Training Accuracy after Epoch 98 : 75.9%
Epoch 99: training loss = 1.197438665002981
Training Accuracy after Epoch 99 : 76.07%
Epoch 100: training loss = 1.1922244670472606
Training Accuracy after Epoch 100 : 76.24%


### Evaluating on Validation data using configuration 1:

In [30]:
labely_test,labelpred=testing(W2,b2,W1,b1,X_testval,y_testval)
accuracy_val=accuracy(labely_test, labelpred)
print("Accuracy on Validation data: {}%".format(accuracy_val))

Accuracy on Validation data: 75.3%
